# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using AtomsBuilder
using DFTK
using LinearAlgebra
using PseudoPotentialData

pseudopotentials = PseudoFamily("dojo.nc.sr.pbesol.v0_4_1.standard.upf")
model = model_DFT(bulk(:Si); functionals=PBEsol(), pseudopotentials)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.397850398111                   -0.90    5.2   27.0ms
  2   -8.400256071796       -2.62       -1.74    1.0   18.7ms
  3   -8.400406663600       -3.82       -2.96    1.5   19.8ms
  4   -8.400427833193       -4.67       -2.98    3.0   57.9ms
  5   -8.400427949655       -6.93       -3.05    1.0   18.9ms
  6   -8.400428149316       -6.70       -4.70    1.0   19.0ms
  7   -8.400428155862       -8.18       -4.45    3.2   26.0ms
  8   -8.400428156256       -9.40       -5.21    1.0   19.6ms
  9   -8.400428156275      -10.72       -6.13    1.0   19.7ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -8.397726925764                   -0.90           4.8    553ms
  2   -8.400374639656       -2.58       -1.77   0.80    2.0    495ms
  3   -8.400422170514       -4.32       -3.00   0.80    1.0    180ms
  4   -8.400428104597       -5.23       -3.46   0.80    2.2   19.9ms
  5   -8.400428151685       -7.33       -4.97   0.80    1.2   17.8ms
  6   -8.400428156269       -8.34       -5.35   0.80    3.0   23.0ms
  7   -8.400428156276      -11.12       -6.86   0.80    1.0   16.4ms


## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   +0.926488639176                   -1.09    2.77s
  2   -1.848359491024        0.44       -0.65    106ms
  3   -4.582656986494        0.44       -0.39   39.3ms
  4   -6.169239948250        0.20       -0.47   73.3ms
  5   -7.680944399394        0.18       -0.73   39.6ms
  6   -7.933004251426       -0.60       -1.45   29.2ms
  7   -8.218204309711       -0.54       -1.62   29.0ms
  8   -8.227933182068       -2.01       -1.94   28.8ms
  9   -8.275254161630       -1.32       -1.79   49.0ms
 10   -8.319166652806       -1.36       -1.67   39.4ms
 11   -8.353780606343       -1.46       -1.75   39.5ms
 12   -8.374376330935       -1.69       -2.43   29.0ms
 13   -8.389239941286       -1.83       -2.40   33.4ms
 14   -8.396952636531       -2.11       -2.50   29.1ms
 15   -8.398593985646       -2.78       -2.59   29.0ms
 16   -8.399631212256       -2.98       -3.11   29.0ms
 17   -8.40

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.397856585042                   -0.90    5.0   26.7ms


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -8.400427983085                   -1.78    24.2s
  2   -8.400428156277       -6.76       -4.02    2.19s
  3   -8.400428156277      -14.75       -7.82    115ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 1.318651071213495e-6
|ρ_newton - ρ_scfv| = 1.2366889710809778e-7
|ρ_newton - ρ_dm|   = 1.5628380220615619e-6
